## **Scraping Komentar Video Youtube Bobon vs Willie**
- Muhammad Habibulloh
- link youtube yang di scraping https://www.youtube.com/watch?v=ygcodv_xrUE

## Import Library

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
# Untuk membuat koneksi ke API Google

## Fungsi Scraping Komentar

In [ ]:
def video_comments(video_id):
    # storing reply
    replies = []

    # Creating youtube resource object dengan API versi v3
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Handle
    video_ids = [video_id] if isinstance(video_id, str) else video_id

    for vid in video_ids:  # Loop agar bisa multi video
        video_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=vid,
            maxResults=100
        ).execute()

        # Iterate video response
        while video_response:
            for item in video_response['items']:
                # Extracting comments
                published = item['snippet']['topLevelComment']['snippet']['publishedAt']
                user = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                likeCount = item['snippet']['topLevelComment']['snippet']['likeCount']

                replies.append([published, vid, user, comment, likeCount])

                # Reply count
                replycount = item['snippet']['totalReplyCount']
                if replycount>0:
                    for reply in item['replies']['comments']:
                        published = reply['snippet']['publishedAt']
                        user = reply['snippet']['authorDisplayName']
                        repl = reply['snippet']['textDisplay']
                        likeCount = reply['snippet']['likeCount']
                        replies.append([published, vid, user, repl, likeCount])

            if 'nextPageToken' in video_response:
                video_response = youtube.commentThreads().list(
                    part='snippet,replies',
                    pageToken=video_response['nextPageToken'],
                    videoId=vid
                ).execute()
            else:
                break

    return replies

## Proses Scraping

In [ ]:
# API Youtube key versi v3
api_key = 'key api taruh sini'

# Daftar video yang akan di scraping (Saya buat multiple scraping video agar suatu saat bisa berguna)
# contoh url video = https://youtu.be/ygcodv_xrUE?feature=shared (bobon vs willie)
video_ids = [
    "ygcodv_xrUE",   # Id Video 1
    #"Video_id2",    # Id Video 2
    #"Video_id3"     # Id Video 3
]

comments = video_comments(video_ids)
comments

[['2025-03-27T16:48:21Z',
  'ygcodv_xrUE',
  '@SteveDunham1312',
  'Dalam hati bobon, wili GOBLOK',
  0],
 ['2025-03-27T16:47:36Z',
  'ygcodv_xrUE',
  '@Arfannn-o9f',
  'njir gaya nya bobon ngotak amat ngomong sm wely seolah dia hakim wily terdakwa..!! woi bon udh bagus wily buat konten berbagi masak besar di pelambng.. kok ini lo malah nge judge gak pake hati lah.. setingan lah.. cm konten lah.. gak ada persiapan lah.. gak ada respect lo jd org..!! mw apa kata lo ngomg yg jelas dia dsna emg berbagi masak besar di palembang.. dasar goblokkk udh kyak ngerasa konten kreator paling bnr paling bagus aja lo.. ngaca woi bon konten dl kyak apa gak jauh2 dari mabok amer, becon, lc dll wkwkkw kocak lo bon so paling tua sok paling senior sok paling bnr..!!',
  0],
 ['2025-03-27T16:46:00Z',
  'ygcodv_xrUE',
  '@Test.tv.tester',
  'Saya cuma minya willie mengakui salah ini settingan , agar menampar semua netizen yg sdh menghujat kota palembang scara tdk langsung',
  1],
 ['2025-03-27T16:45:45Z',
 

## Ubah Hasil Scraping ke Dataframe

In [ ]:
df = pd.DataFrame(comments, columns=[
    'publishedAt',
    'videoId',
    'authorDisplayName',
    'textDisplay',
    'likeCount'
])

df

,publishedAt,videoId,authorDisplayName,textDisplay,likeCount
0,2025-03-27T16:48:21Z,ygcodv_xrUE,@SteveDunham1312,"Dalam hati bobon, wili GOBLOK",0
1,2025-03-27T16:47:36Z,ygcodv_xrUE,@Arfannn-o9f,njir gaya nya bobon ngotak amat ngomong sm wel...,0
2,2025-03-27T16:46:00Z,ygcodv_xrUE,@Test.tv.tester,Saya cuma minya willie mengakui salah ini sett...,1
3,2025-03-27T16:45:45Z,ygcodv_xrUE,@nunikariyyan7232,Bang bobon dan Willy Salim sama sama ora yg ba...,0
4,2025-03-27T16:43:26Z,ygcodv_xrUE,@Bangziii15,oke,0
...,...,...,...,...,...
24387,2025-03-25T04:31:14Z,ygcodv_xrUE,@teguhwidodo6793,Serem nihh,0
24388,2025-03-25T05:13:06Z,ygcodv_xrUE,@BabiKon-s4q,Malah flexing si wili lg njir😅😅😅 pdhl ksmptn d...,0
24389,2025-03-25T04:31:09Z,ygcodv_xrUE,@shyvasanusi3595,❤,1
24390,2025-03-25T04:31:04Z,ygcodv_xrUE,@SholihinLihin-s3n,Pertama,0


## Simpan ke file CSV

In [ ]:
df.to_csv('ytb-bobonvswillie.csv', index=False)